In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
import textwrap
import keras
import random
import pandas as pd
from transformers import AdamW, get_linear_schedule_with_warmup, AutoTokenizer, AutoModelForSequenceClassification
import progressbar
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import numpy as np
import time
import os
from sklearn.metrics import confusion_matrix

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.1 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Thesis/train_data/ILDC_multi.csv')

df_train = df.query(" split=='train' ")
df_test = df.query(" split=='test' ")
df_dev = df.query(" split=='dev' ")

In [ ]:
output_dir = '/content/drive/MyDrive/Thesis/calibration/saved_model_multi_2560'
device = torch.device('cuda')
model = AutoModelForSequenceClassification.from_pretrained(output_dir, output_hidden_states=True, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(output_dir)
model.to(device)

LSGBertForSequenceClassification(
  (bert): LSGBertModel(
    (embeddings): LSGBertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (global_embeddings): Embedding(512, 768)
    )
    (encoder): LSGBertEncoder(
      (layer): ModuleList(
        (0-11): 12 x LSGBertLayer(
          (attention): LSGAttention(
            (self): LSGSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (attention): LSGAttentionProduct(
                (attention): BaseAttentionProduct(
                  (drop

In [ ]:
def att_masking(input_ids):
  attention_masks = []
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_masks.append(att_mask)
  return attention_masks

In [ ]:
def grouped_input_ids_reverse(all_toks):
    splitted_toks = []
    total_tokens = len(all_toks)
    chunk_size = 510
    overlap_size = 100

    end = total_tokens
    if (end<101):
      splitted_toks.insert(0, all_toks)
    else:
      while end > 100:
        start = max(0, end - chunk_size)
        chunk = all_toks[start:end]
        splitted_toks.insert(0, chunk)
        if start!=0:
          end = start + overlap_size
        else:
          end=0

    CLS = tokenizer.cls_token
    SEP = tokenizer.sep_token
    e_sents = []
    for chunk in splitted_toks:
        chunk = [CLS] + chunk + [SEP]
        encoded_sent = tokenizer.convert_tokens_to_ids(chunk)
        e_sents.append(encoded_sent)

    e_sents = pad_sequences(e_sents, maxlen=512, value=0, dtype="long", padding="pre")
    att_masks = att_masking(e_sents)
    return e_sents, att_masks


In [ ]:
def get_output_for_one_vec(input_id, att_mask):
  input_ids = torch.tensor(input_id)
  att_masks = torch.tensor(att_mask)
  input_ids = input_ids.unsqueeze(0)
  att_masks = att_masks.unsqueeze(0)
  model.eval()
  input_ids = input_ids.to(device)
  att_masks = att_masks.to(device)
  with torch.no_grad():
      outputs = model(input_ids=input_ids, attention_mask=att_masks)

  vec = outputs["hidden_states"][12][0][0]
  vec = vec.detach().cpu().numpy()
  return vec

In [ ]:
def generate_np_files_for_emb(dataf, tokenizer):
  all_docs = []
  for i in progressbar.progressbar(range(len(dataf['text']))):
    text = dataf['text'].iloc[i]
    toks = tokenizer.tokenize(text)
    if(len(toks) > 10000):
      toks = toks[len(toks)-10000:]

    splitted_input_ids, splitted_att_masks = grouped_input_ids_reverse(toks)

    vecs = []
    for index,ii in enumerate(splitted_input_ids):
      vecs.append(get_output_for_one_vec(ii, splitted_att_masks[index]))

    one_doc = np.asarray(vecs)
    all_docs.append(one_doc)

  all_docs = np.asarray(all_docs)
  return all_docs

In [ ]:
# CLS
path_val_npy_file = "/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation/LSGBERT_bigru_att_full/LSGBERT_npy_files_cls_multi_full/LSGBERT_cls_train"
vecs_train = generate_np_files_for_emb(df_train, tokenizer)
np.save(path_val_npy_file, vecs_train)

print('npy file train saved')

  0% (0 of 32305) |                      | Elapsed Time: 0:00:00 ETA:  --:--:--Token indices sequence length is longer than the specified maximum sequence length for this model (55443 > 4096). Running this sequence through the model will result in indexing errors
 93% (30266 of 32305) |################  | Elapsed Time: 3:39:11 ETA:   0:05:19

In [ ]:
# CLS
path_val_npy_file = "/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation/LSGBERT_bigru_att_full/LSGBERT_npy_files_cls_multi_full/LSGBERT_cls_dev"
vecs_dev = generate_np_files_for_emb(df_dev, tokenizer)
np.save(path_val_npy_file, vecs_dev)

print('npy file dev saved')

  0% (0 of 994) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--Token indices sequence length is longer than the specified maximum sequence length for this model (7726 > 4096). Running this sequence through the model will result in indexing errors
100% (994 of 994) |######################| Elapsed Time: 0:08:45 Time:  0:08:45
<ipython-input-8-c2dcbbfdf16d>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_docs = np.asarray(all_docs)


npy file dev saved


In [ ]:
# CLS
path_val_npy_file = "/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation/LSGBERT_bigru_att_full/LSGBERT_npy_files_cls_multi_full/LSGBERT_cls_test"
vecs_test = generate_np_files_for_emb(df_test, tokenizer)
np.save(path_val_npy_file, vecs_test)

print('npy file test saved')

100% (1517 of 1517) |####################| Elapsed Time: 0:13:49 Time:  0:13:49


npy file test saved


<ipython-input-8-c2dcbbfdf16d>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_docs = np.asarray(all_docs)
